In [9]:
import pandas as pd
from consts import PATH, TEST_PATH, TRAIN_PATH
from train_test_data_split import TrainTestSplitter

In [10]:
TrainTestSplitter(left_part_days=180).save_splited_rfm()

21:41:38 Save splitted rfm
21:41:38 Getting splited alive partners rfm
21:41:38 Alive partners calculation


KeyboardInterrupt: 

In [12]:
new_data = pd.read_parquet("data/train1.parquet.gzip")
new_data

,monetary_value,first_buy,last_buy,count,alive,frequency,recency,T
partner,,,,,,,,
0,5.698337e-10,2021-08-27,2022-01-27,8,True,7,153,545
1,1.712412e-10,2021-08-23,2022-01-27,17,True,16,157,549
2,2.016771e-09,2021-09-25,2021-09-25,1,True,0,0,516
4,1.748972e-09,2021-10-19,2022-02-04,5,False,4,108,492
9,1.858965e-10,2021-08-22,2022-01-10,7,True,6,141,550
...,...,...,...,...,...,...,...,...
1229908,1.445898e-12,2022-02-01,2022-02-01,1,True,0,0,387
1229910,6.024121e-14,2022-02-01,2022-02-01,1,True,0,0,387
1229914,4.461669e-13,2021-10-25,2022-02-01,2,False,1,99,486


In [13]:
(new_data['first_buy'].max() - new_data['first_buy'].min()).days

179

In [14]:
def calc_destribution(data: dict) -> dict:
    data_sum = sum(data.values())
    distribution = {}
    for key, value in data.items():
        distribution[key] = f"{round(value / data_sum, 3) * 100:.1f}%"
    return distribution

In [19]:
from sklearn.metrics import recall_score, f1_score, precision_score, roc_auc_score

In [39]:
new_data["alive"].value_counts()
new_data = new_data[new_data["alive"]==True].sample(96991)
new_data["alive"].unique()
new_data = pd.concat([new_data, new_data[new_data["alive"]==False]])
new_data['alive'].value_counts()

KeyError: 'alive'

In [33]:
for day in range(30,91):
    last_data_date = new_data['first_buy'].max()
    new_data['pred_alive'] = new_data['last_buy'].apply(lambda x: (last_data_date - x).days < day)
    recall = recall_score(new_data['alive'], new_data['pred_alive'])
    f1 = f1_score(new_data['alive'], new_data['pred_alive'])
    precision = precision_score(new_data['alive'], new_data['pred_alive'])
    roc_auc = roc_auc_score(new_data['alive'], new_data['pred_alive'])
    print(f"Day {day}: recall: {recall:.3f}, f1: {f1:.3f}, precision: {precision:.3f}, roc_auc: {roc_auc:.3f}")

Day 30: recall: 0.742, f1: 0.755, precision: 0.767, roc_auc: 0.759
Day 31: recall: 0.745, f1: 0.755, precision: 0.766, roc_auc: 0.758
Day 32: recall: 0.746, f1: 0.756, precision: 0.765, roc_auc: 0.759
Day 33: recall: 0.750, f1: 0.756, precision: 0.763, roc_auc: 0.758
Day 34: recall: 0.752, f1: 0.756, precision: 0.761, roc_auc: 0.758
Day 35: recall: 0.754, f1: 0.757, precision: 0.760, roc_auc: 0.758
Day 36: recall: 0.756, f1: 0.757, precision: 0.759, roc_auc: 0.758
Day 37: recall: 0.757, f1: 0.757, precision: 0.758, roc_auc: 0.758
Day 38: recall: 0.758, f1: 0.758, precision: 0.757, roc_auc: 0.757
Day 39: recall: 0.762, f1: 0.758, precision: 0.755, roc_auc: 0.757
Day 40: recall: 0.772, f1: 0.761, precision: 0.750, roc_auc: 0.757
Day 41: recall: 0.784, f1: 0.765, precision: 0.746, roc_auc: 0.758
Day 42: recall: 0.794, f1: 0.766, precision: 0.741, roc_auc: 0.758
Day 43: recall: 0.803, f1: 0.768, precision: 0.736, roc_auc: 0.757
Day 44: recall: 0.805, f1: 0.768, precision: 0.735, roc_auc: 0

In [22]:
new_data['True']=True
recall = recall_score(new_data['alive'], new_data['True'])
f1 = f1_score(new_data['alive'], new_data['True'])
precision = precision_score(new_data['alive'], new_data['True'])
roc_auc = roc_auc_score(new_data['alive'], new_data['True'])
print(f"recall: {recall:.3f}, f1: {f1:.3f}, precision: {precision:.3f}, roc_auc: {roc_auc:.3f}")

recall: 1.000, f1: 0.838, precision: 0.721, roc_auc: 0.500


In [61]:
from btyd.utils import calculate_alive_path, expected_cumulative_transactions
import pandas as pd
import numpy as np
import pickle

from consts import MODEL_PATH_180_DAYS, TRAIN_RAW_PATH, TEST_RAW_PATH, TEST_PATH, TRAIN_PATH

data = pd.read_parquet(TRAIN_RAW_PATH)
rfm = pd.read_parquet("data/train_180.parquet.gzip")
model = pickle.load(open(MODEL_PATH_180_DAYS, "rb"))

rfm_100 = rfm.sample(100)


for partner in rfm_100.index:
    t = rfm["T"].max()
    transactions = data[data["partner"]==partner]
    datetime_col = "rep_date"
    freq="D"
    start_date=None
    ax=None

    customer_history = transactions[[datetime_col]].copy()
    customer_history.index = pd.DatetimeIndex(customer_history[datetime_col])

    # Add transactions column
    customer_history["transactions"] = 1
    customer_history = customer_history.resample(freq).sum()
    # plot alive_path
    path = calculate_alive_path(model, transactions, datetime_col, t, freq)
    path_dates = pd.date_range(start=min(transactions[datetime_col]), periods=len(path), freq=freq)
    rfm_100.loc[partner]["alive_prob"] = path.iloc[-1]

ValueError: min() arg is an empty sequence

In [69]:
data["partner"].nunique()

444541

In [68]:
rfm.count()

monetary_value    347497
first_buy         347497
last_buy          347497
count             347497
alive             347497
frequency         347497
recency           347497
T                 347497
dtype: int64

In [59]:
data[data["partner"]==399080]

,partner,rep_date,monetary,is_alive


In [49]:
new_data["rep_date"].max()

datetime.date(2021, 12, 15)

In [50]:
path_dates

DatetimeIndex(['2021-09-12', '2021-09-13', '2021-09-14', '2021-09-15',
               '2021-09-16', '2021-09-17', '2021-09-18', '2021-09-19',
               '2021-09-20', '2021-09-21',
               ...
               '2022-06-05', '2022-06-06', '2022-06-07', '2022-06-08',
               '2022-06-09', '2022-06-10', '2022-06-11', '2022-06-12',
               '2022-06-13', '2022-06-14'],
              dtype='datetime64[ns]', length=276, freq='D')

In [51]:
path

0                       [1.0]
1                       [1.0]
2                       [1.0]
3                       [1.0]
4                       [1.0]
                ...          
271    [0.010697097808025224]
272     [0.01049327365682833]
273    [0.010293994911447994]
274     [0.01009914616067771]
275    [0.009908615254858292]
Length: 276, dtype: object

In [52]:
from tqdm import tqdm
import time
for i in tqdm(range(100)):
    time.sleep(1)

  1%|          | 1/100 [00:02<03:18,  2.00s/it]


KeyboardInterrupt: 